### 공고데이터셋 채용공고, 자격요건및우대사항 요약

In [66]:
# make a folder
folderpath = '.\\result'
makedirs(folderpath)

path = '..\\gongodataset\\gongo_byjob\\'
file_list = os.listdir(path)
file_list_name = [file for file in file_list if file.endswith('.csv')]

# by job
for filename in file_list_name[5:]: # modified
	# read df
	df = pd.read_csv('{}{}'.format(path, filename))

	# drop nan
	df.dropna(subset = ['주요 업무','자격요건 및 우대사항'], inplace=True)

	# data
	work = df['주요 업무']
	qual = df['자격요건 및 우대사항']

	# make 1 str type
	work_sen_all = '\n'.join(work)
	qual_sen_all = '\n'.join(qual)

	# model
	model_dir = "lcw99/t5-large-korean-text-summary"
	tokenizer = AutoTokenizer.from_pretrained(model_dir)
	model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

	# parameters
	max_input_length = 512

	# summarize work sentences
	inputs_w = ["summarize_summary: " + work_sen_all]  # summarize
	inputs_w = tokenizer(inputs_w, max_length=max_input_length, truncation=True, return_tensors="pt")
	output_w = model.generate(**inputs_w, num_beams=8, do_sample=True, min_length=10, max_length=300)
	decoded_output_w = tokenizer.batch_decode(output_w, skip_special_tokens=True)[0]
	predicted_w = nltk.sent_tokenize(decoded_output_w.strip())[0]

	# summarize qualification sentences
	inputs_q = ["summarize_summary: " + qual_sen_all]  # summarize
	inputs_q = tokenizer(inputs_q, max_length=max_input_length, truncation=True, return_tensors="pt")
	output_q = model.generate(**inputs_q, num_beams=8, do_sample=True, min_length=10, max_length=300)
	decoded_output_q = tokenizer.batch_decode(output_q, skip_special_tokens=True)[0]
	predicted_q = nltk.sent_tokenize(decoded_output_q.strip())[0]

	# save a predicted sentence as a text file
	with open('{}\\{}_work.txt'.format(folderpath,filename.replace('.csv','')),'w') as f:
		f.write(predicted_w)
	f.close()

	with open('{}\\{}_qual.txt'.format(folderpath,filename.replace('.csv','')),'w') as f:
		f.write(predicted_q)
	f.close()